# Array Pattern Synthesis with PyAEDT

This notebook demonstrates automated pattern synthesis for phased arrays using PyAEDT.
We extract port positions from HFSS geometry and implement beam steering algorithms.

## Key Capabilities:
- ✅ Extract element positions from lumped port geometry
- ✅ Variable-based excitation management 
- ✅ Beam steering calculations
- ✅ Array geometry analysis

## 1. Setup and Connection

In [37]:
# Core imports
from ansys.aedt.core import Hfss
import numpy as np
import matplotlib.pyplot as plt
import os

print("📦 Imports successful!")

📦 Imports successful!


In [38]:
# Project configuration
PROJECT_PATH = r"C:\Mac\Home\Documents\HFSS_Projects\array_example\planar_flared_dipole_array.aedt"
AEDT_VERSION = "2024.2"
NON_GRAPHICAL = False

print(f"🔧 Configuration:")
print(f"  Project: {PROJECT_PATH}")
print(f"  Version: {AEDT_VERSION}")
print(f"  Graphical: {not NON_GRAPHICAL}")

if os.path.exists(PROJECT_PATH):
    print(f"  ✅ Project file found")
else:
    print(f"  ❌ Project file not found - update PROJECT_PATH")

🔧 Configuration:
  Project: C:\Mac\Home\Documents\HFSS_Projects\array_example\planar_flared_dipole_array.aedt
  Version: 2024.2
  Graphical: True
  ✅ Project file found


In [39]:
# Connect to HFSS
try:
    print("🔌 Connecting to HFSS...")
    
    hfss = Hfss(
        project=PROJECT_PATH,
        version=AEDT_VERSION,
        non_graphical=NON_GRAPHICAL,
        student_version=True,
        remove_lock=True
    )
    
    print(f"✅ Connected successfully!")
    print(f"   Project: {hfss.project_name}")
    print(f"   Design: {hfss.design_name}")
    print(f"   Type: {hfss.design_type}")
    print(f"   Solution: {hfss.solution_type}")
    
    # Quick validation
    ports = hfss.ports
    print(f"   Ports: {len(ports)} → {ports}")
    
except Exception as e:
    print(f"❌ Connection failed: {e}")
    hfss = None

🔌 Connecting to HFSS...
PyAEDT INFO: Python version 3.12.9 | packaged by Anaconda, Inc. | (main, Feb  6 2025, 18:49:16) [MSC v.1929 64 bit (AMD64)].
PyAEDT INFO: Parsing C:\Mac\Home\Documents\HFSS_Projects\array_example\planar_flared_dipole_array.aedt.
PyAEDT INFO: PyAEDT version 0.15.0.
PyAEDT INFO: Returning found Desktop session with PID 15140!
PyAEDT INFO: Project planar_flared_dipole_array set to active.
PyAEDT INFO: Active Design set to HFSSDesign1
PyAEDT INFO: Active Design set to HFSSDesign1
PyAEDT INFO: File C:\Mac\Home\Documents\HFSS_Projects\array_example\planar_flared_dipole_array.aedt correctly loaded. Elapsed time: 0m 0sec
PyAEDT INFO: Aedt Objects correctly read
✅ Connected successfully!
   Project: planar_flared_dipole_array
   Design: HFSSDesign1
   Type: HFSS
   Solution: Modal
   Ports: 5 → ['1', '2', '3', '4', '5']


## 2. Extract Port Positions

Extract element positions from lumped port geometry using the validated strategy.

In [40]:
def extract_port_positions(hfss):
    """Extract port positions from lumped port geometry."""
    
    if hfss is None:
        print("❌ No HFSS connection")
        return {}
    
    print("🎯 Extracting Port Positions...")
    
    port_positions = {}
    boundaries = hfss.boundaries
    
    for boundary in boundaries:
        try:
            boundary_name = getattr(boundary, 'name', 'Unknown')
            boundary_type = getattr(boundary, 'type', 'Unknown')
            
            if 'port' in str(boundary_type).lower():
                print(f"  📍 Port {boundary_name}:", end=" ")
                
                if hasattr(boundary, 'props') and boundary.props:
                    props = boundary.props
                    
                    # Access lumped port geometry
                    if ('Modes' in props and 'Mode1' in props['Modes'] and 
                        'IntLine' in props['Modes']['Mode1'] and 
                        'GeometryPosition' in props['Modes']['Mode1']['IntLine']):
                        
                        geo_positions = props['Modes']['Mode1']['IntLine']['GeometryPosition']
                        positions = []
                        
                        for pos in geo_positions:
                            x = float(pos['XPosition'])
                            y = float(pos['YPosition'])
                            z = float(pos['ZPosition'])
                            positions.append([x, y, z])
                        
                        # Calculate port position (midpoint for multi-point ports)
                        if len(positions) >= 2:
                            port_pos = np.mean(positions, axis=0)
                        elif len(positions) == 1:
                            port_pos = np.array(positions[0])
                        else:
                            print("No positions found")
                            continue
                        
                        port_positions[boundary_name] = {
                            'position': port_pos.tolist(),
                            'x': port_pos[0],
                            'y': port_pos[1],
                            'z': port_pos[2]
                        }
                        
                        print(f"[{port_pos[0]:.2f}, {port_pos[1]:.2f}, {port_pos[2]:.2f}]")
                    else:
                        print("No geometry data")
                else:
                    print("No properties")
                    
        except Exception as e:
            print(f"  ❌ Error processing {boundary_name}: {e}")
    
    return port_positions

# Extract positions
port_positions = extract_port_positions(hfss)

if port_positions:
    print(f"\n✅ Extracted {len(port_positions)} port positions")
else:
    print(f"\n❌ No port positions extracted")

🎯 Extracting Port Positions...
  📍 Port 1: [-0.00, -200.00, -28.61]
  📍 Port 2: [-0.00, -100.00, -28.61]
  📍 Port 3: [-0.00, 0.00, -28.61]
  📍 Port 4: [-0.00, 100.00, -28.61]
  📍 Port 5: [-0.00, 200.00, -28.61]

✅ Extracted 5 port positions


## 3. Array Geometry Analysis

Analyze array structure and calculate element spacings.

In [41]:
def analyze_array_geometry(port_positions):
    """Analyze array geometry and determine structure."""
    
    if len(port_positions) < 2:
        print("❌ Need at least 2 ports for analysis")
        return None
    
    print(f"📐 Array Geometry Analysis ({len(port_positions)} elements):")
    
    # Extract positions and port names
    positions = np.array([info['position'] for info in port_positions.values()])
    port_names = list(port_positions.keys())
    
    # Calculate array extents
    x_extent = np.max(positions[:, 0]) - np.min(positions[:, 0])
    y_extent = np.max(positions[:, 1]) - np.min(positions[:, 1])
    z_extent = np.max(positions[:, 2]) - np.min(positions[:, 2])
    
    extents = [x_extent, y_extent, z_extent]
    axis_names = ['X', 'Y', 'Z']
    primary_axis_idx = np.argmax(extents)
    primary_axis = axis_names[primary_axis_idx]
    
    print(f"  📏 Extents: X={x_extent:.2f}, Y={y_extent:.2f}, Z={z_extent:.2f} mm")
    print(f"  🧭 Primary axis: {primary_axis}")
    
    # Determine if linear array
    secondary_extents = [ext for i, ext in enumerate(extents) if i != primary_axis_idx]
    max_secondary = max(secondary_extents)
    is_linear = max_secondary < (extents[primary_axis_idx] * 0.1)
    
    array_info = {
        'positions': positions,
        'port_names': port_names,
        'extents': {'X': x_extent, 'Y': y_extent, 'Z': z_extent},
        'primary_axis': primary_axis,
        'is_linear': is_linear
    }
    
    if is_linear:
        print(f"  📡 Array type: LINEAR along {primary_axis}-axis")
        
        # Sort elements along primary axis
        sort_coords = positions[:, primary_axis_idx]
        sorted_indices = np.argsort(sort_coords)
        
        # Calculate adjacent spacings
        spacings = []
        print(f"  🔗 Element order and spacings:")
        
        for i, idx in enumerate(sorted_indices):
            port_name = port_names[idx]
            pos = positions[idx]
            
            if i > 0:
                prev_idx = sorted_indices[i-1]
                spacing = np.linalg.norm(positions[idx] - positions[prev_idx])
                spacings.append(spacing)
                print(f"    {i+1}. {port_name} (spacing: {spacing:.2f} mm)")
            else:
                print(f"    {i+1}. {port_name} (reference)")
        
        if spacings:
            avg_spacing = np.mean(spacings)
            spacing_std = np.std(spacings)
            is_uniform = spacing_std / avg_spacing < 0.05  # 5% tolerance
            
            print(f"  📊 Average spacing: {avg_spacing:.2f} mm")
            print(f"  ✨ Uniform spacing: {'Yes' if is_uniform else 'No'}")
            
            array_info.update({
                'sorted_indices': sorted_indices,
                'spacings': spacings,
                'avg_spacing': avg_spacing,
                'is_uniform': is_uniform
            })
    else:
        print(f"  📡 Array type: PLANAR/3D")
    
    return array_info

# Analyze geometry
if port_positions:
    array_info = analyze_array_geometry(port_positions)
else:
    array_info = None

📐 Array Geometry Analysis (5 elements):
  📏 Extents: X=0.00, Y=400.00, Z=0.00 mm
  🧭 Primary axis: Y
  📡 Array type: LINEAR along Y-axis
  🔗 Element order and spacings:
    1. 1 (reference)
    2. 2 (spacing: 100.00 mm)
    3. 3 (spacing: 100.00 mm)
    4. 4 (spacing: 100.00 mm)
    5. 5 (spacing: 100.00 mm)
  📊 Average spacing: 100.00 mm
  ✨ Uniform spacing: Yes


## 4. Excitation Management System

Set up variable-based excitation control for each port.

In [ ]:
class ArrayExcitationManager:
    """Manages port excitations using HFSS post-processing variables (OPTIMAL pattern)."""
    
    def __init__(self, hfss, port_positions):
        self.hfss = hfss
        self.port_positions = port_positions
        self.ports = list(port_positions.keys())
        self.excitation_vars = {}
        
        print(f"🎛️  Initializing excitation manager for {len(self.ports)} ports")
        self._setup_variables()
        self._apply_to_sources()
        
    def _setup_variables(self):
        """Create HFSS post-processing variables for each port."""
        
        for i, port in enumerate(self.ports):
            mag_var = f"Port{port}_Magnitude"  # Use "Port" prefix for numeric port names
            phase_var = f"Port{port}_Phase"
            
            # Store variable names
            self.excitation_vars[port] = {
                'magnitude_var': mag_var,
                'phase_var': phase_var
            }
            
            # Create POST-PROCESSING variables (first port active, others off)
            if i == 0:
                success1 = self.hfss.variable_manager.set_variable(mag_var, "1W", is_post_processing=True)
                success2 = self.hfss.variable_manager.set_variable(phase_var, "0deg", is_post_processing=True)
                print(f"  ✅ {port}: {mag_var}=1W ({success1}), {phase_var}=0deg ({success2}) (active)")
            else:
                success1 = self.hfss.variable_manager.set_variable(mag_var, "0W", is_post_processing=True)
                success2 = self.hfss.variable_manager.set_variable(phase_var, "0deg", is_post_processing=True)
                print(f"  💤 {port}: {mag_var}=0W ({success1}), {phase_var}=0deg ({success2}) (inactive)")
    
    def _apply_to_sources(self):
        """Apply post-processing variables to HFSS sources WITHOUT $ prefix."""
        print(f"🔗 Linking variables to HFSS sources...")
        
        # Build sources dictionary WITHOUT $ prefix (post-processing variables)
        sources = {}
        for port in self.ports:
            mag_var = self.excitation_vars[port]['magnitude_var']
            phase_var = self.excitation_vars[port]['phase_var']
            sources[f"{port}:1"] = (mag_var, phase_var)  # NO $ prefix!
            print(f"  📡 {port}:1 → {mag_var}, {phase_var}")
        
        try:
            self.hfss.edit_sources(sources)
            print(f"  ✅ Sources linked to post-processing variables")
            print(f"  🚀 Variables can now be updated for real-time tuning!")
        except Exception as e:
            print(f"  ❌ Failed to link sources: {e}")
    
    def get_excitations(self):
        """Get current excitation values from post-processing variables."""
        excitations = {}
        
        try:
            post_vars = self.hfss.variable_manager.post_processing_variables
            
            for port in self.ports:
                mag_var = self.excitation_vars[port]['magnitude_var']
                phase_var = self.excitation_vars[port]['phase_var']
                
                if mag_var in post_vars and phase_var in post_vars:
                    excitations[port] = {
                        'magnitude': post_vars[mag_var].evaluated_value,
                        'phase': post_vars[phase_var].evaluated_value
                    }
                else:
                    excitations[port] = {'magnitude': '0W', 'phase': '0deg'}
                    
        except Exception as e:
            print(f"❌ Error reading excitations: {e}")
        
        return excitations
    
    def set_excitations(self, excitation_dict):
        """Set new excitation values in post-processing variables (real-time update)."""
        
        print(f"🔄 Updating post-processing variables...")
        
        success_count = 0
        for port, values in excitation_dict.items():
            if port in self.excitation_vars:
                mag_var = self.excitation_vars[port]['magnitude_var']
                phase_var = self.excitation_vars[port]['phase_var']
                
                try:
                    # Update post-processing variables - HFSS automatically applies changes!
                    success1 = self.hfss.variable_manager.set_variable(
                        mag_var, values['magnitude'], is_post_processing=True
                    )
                    success2 = self.hfss.variable_manager.set_variable(
                        phase_var, values['phase'], is_post_processing=True
                    )
                    
                    if success1 and success2:
                        print(f"  📡 {port}: {values['magnitude']}, {values['phase']}")
                        success_count += 1
                    else:
                        print(f"  ⚠️  {port}: variable update issues ({success1}, {success2})")
                    
                except Exception as e:
                    print(f"  ❌ Failed to update {port}: {e}")
        
        if success_count == len(excitation_dict):
            print(f"  ✅ All variables updated successfully!")
            print(f"  🚀 HFSS automatically uses new values (no edit_sources needed)")
            return True
        else:
            print(f"  ⚠️  Updated {success_count}/{len(excitation_dict)} variables")
            return False
    
    def print_status(self):
        """Display current excitation status."""
        print(f"\n📊 Current Excitation Status:")
        current = self.get_excitations()
        for port, values in current.items():
            print(f"  {port}: {values['magnitude']}, {values['phase']}")
        
        print(f"\n🔗 Variable Linkage (post-processing, no $ prefix):")
        for port in self.ports:
            mag_var = self.excitation_vars[port]['magnitude_var']
            phase_var = self.excitation_vars[port]['phase_var']
            print(f"  {port}:1 ← {mag_var}, {phase_var}")

# Initialize excitation manager
if hfss is not None and port_positions:
    excitation_mgr = ArrayExcitationManager(hfss, port_positions)
    excitation_mgr.print_status()
else:
    print("❌ Cannot initialize excitation manager")

## 5. Beam Steering Implementation

Calculate and apply phase shifts for beam steering using actual element positions.

In [ ]:
def calculate_beam_steering_phases(port_positions, array_info, theta_deg, frequency_ghz=77):
    """Calculate phase shifts for beam steering."""
    
    if array_info is None or not array_info['is_linear']:
        print("❌ Beam steering requires linear array")
        return None
    
    print(f"🎯 Calculating beam steering phases:")
    print(f"  Target angle: {theta_deg}°")
    print(f"  Frequency: {frequency_ghz} GHz")
    
    # Physical constants
    c = 3e8  # Speed of light (m/s)
    wavelength = c / (frequency_ghz * 1e9)  # meters
    k = 2 * np.pi / wavelength  # wavenumber
    
    print(f"  Wavelength: {wavelength*1000:.2f} mm")
    
    # Get element positions and sort by primary axis
    positions = array_info['positions']
    sorted_indices = array_info['sorted_indices']
    port_names = array_info['port_names']
    primary_axis = array_info['primary_axis']
    
    # Reference position (first element)
    ref_idx = sorted_indices[0]
    ref_position = positions[ref_idx]
    
    excitations = {}
    
    print(f"\n📡 Element phases (relative to {port_names[ref_idx]}):")
    
    for i, idx in enumerate(sorted_indices):
        port_name = port_names[idx]
        element_pos = positions[idx]
        
        # Calculate distance along primary axis from reference
        if primary_axis == 'X':
            distance = (element_pos[0] - ref_position[0]) / 1000  # convert mm to m
        elif primary_axis == 'Y':
            distance = (element_pos[1] - ref_position[1]) / 1000  # convert mm to m
        else:  # Z
            distance = (element_pos[2] - ref_position[2]) / 1000  # convert mm to m
        
        # Calculate phase shift for beam steering
        # Phase = -k * d * sin(theta) where d is element spacing from reference
        phase_rad = -k * distance * np.sin(np.radians(theta_deg))
        phase_deg = np.degrees(phase_rad)
        
        # Wrap phase to [-180, 180] degrees
        phase_deg = ((phase_deg + 180) % 360) - 180
        
        excitations[port_name] = {
            'magnitude': '1W',
            'phase': f'{phase_deg:.1f}deg'
        }
        
        print(f"  {port_name}: distance={distance*1000:.1f}mm, phase={phase_deg:.1f}°")
    
    return excitations

def apply_beam_steering(excitation_mgr, array_info, theta_deg, frequency_ghz=77):
    """Apply beam steering to the array."""
    
    # Calculate phases
    excitations = calculate_beam_steering_phases(
        excitation_mgr.port_positions, array_info, theta_deg, frequency_ghz
    )
    
    if excitations:
        # Apply to HFSS
        success = excitation_mgr.set_excitations(excitations)
        
        if success:
            print(f"\n✅ Beam steering applied successfully!")
            print(f"   Array steered to {theta_deg}° at {frequency_ghz} GHz")
        else:
            print(f"\n❌ Failed to apply beam steering")
    
    return excitations

# Test beam steering
if 'excitation_mgr' in locals() and array_info and array_info['is_linear']:
    print("🎯 Testing Beam Steering:")
    
    # Steer to 30 degrees
    test_angle = 30
    beam_excitations = apply_beam_steering(excitation_mgr, array_info, test_angle)
    
else:
    print("❌ Beam steering not available (requires linear array)")

## 6. Validation and Testing

Validate the complete workflow and test different beam angles.

In [ ]:
# Comprehensive validation
if hfss is not None:
    print("🔍 System Validation:")
    print("=" * 40)
    
    # Check connection
    print(f"✅ HFSS Connection: {hfss.project_name} / {hfss.design_name}")
    
    # Check port extraction
    if port_positions:
        print(f"✅ Port Positions: {len(port_positions)} extracted")
        for port, info in port_positions.items():
            pos = info['position']
            print(f"   {port}: [{pos[0]:.2f}, {pos[1]:.2f}, {pos[2]:.2f}]")
    else:
        print(f"❌ Port Positions: None extracted")
    
    # Check array analysis
    if array_info:
        print(f"✅ Array Analysis: {array_info['primary_axis']}-axis, {'Linear' if array_info['is_linear'] else 'Planar'}")
        if array_info['is_linear']:
            print(f"   Average spacing: {array_info['avg_spacing']:.2f} mm")
    else:
        print(f"❌ Array Analysis: Failed")
    
    # Check excitation manager
    if 'excitation_mgr' in locals():
        print(f"✅ Excitation Manager: {len(excitation_mgr.ports)} ports managed")
    else:
        print(f"❌ Excitation Manager: Not initialized")
    
    # Check beam steering capability
    if array_info and array_info['is_linear']:
        print(f"✅ Beam Steering: Available for linear array")
    else:
        print(f"⚠️  Beam Steering: Requires linear array")
    
    print("\n🎉 Array Pattern Synthesis System Ready!")
    
else:
    print("❌ System validation failed - no HFSS connection")

In [ ]:
# Test multiple beam angles
if 'excitation_mgr' in locals() and array_info and array_info['is_linear']:
    print("🎯 Testing Multiple Beam Angles:")
    print("=" * 35)
    
    test_angles = [0, 15, 30, 45, -15, -30]
    
    for angle in test_angles:
        print(f"\n📐 Steering to {angle:+3d}°:")
        
        # Calculate phases (without applying to HFSS)
        excitations = calculate_beam_steering_phases(
            port_positions, array_info, angle, frequency_ghz=77
        )
        
        if excitations:
            print(f"   Phase summary:")
            for port, vals in excitations.items():
                print(f"   {port}: {vals['phase']}")
    
    print(f"\n💡 To apply any angle, use:")
    print(f"   apply_beam_steering(excitation_mgr, array_info, angle_deg)")
    
else:
    print("❌ Beam angle testing not available")

## 7. Cleanup

In [ ]:
# Uncomment to close HFSS connection
# if hfss is not None:
#     hfss.close()
#     print("🔌 HFSS connection closed")